# reaction-network (Demo Notebook): Enumerators

### Author: Matthew McDermott
Last Updated: 12/07/21

**If you use this code in your work, please consider citing the following paper:**

McDermott, M. J., Dwaraknath, S. S., and Persson, K. A. (2021). A graph-based network for predicting chemical reaction pathways in solid-state materials synthesis. 
Nature Communications, 12(1). https://doi.org/10.1038/s41467-021-23339-x

### Imports

In [1]:
import logging
from pprint import pprint

from pymatgen.core.composition import Composition, Element
from mp_api import MPRester

from rxn_network.enumerators.basic import BasicEnumerator # BasicOpenEnumerator
#from rxn_network.enumerators.minimize import MinimizeGibbsEnumerator, MinimizeGrandPotentialEnumerator
from rxn_network.fireworks import EnumeratorFW
from rxn_network.costs.softplus import Softplus
from rxn_network.entries.entry_set import GibbsEntrySet

%load_ext autoreload
%autoreload 2

## Downloading and modifying entries

First, we acquire entries for phases in the Y-Mn-O chemical system from the Materials Project (MP), a computed materials database containing calculations for over 130,000 materials.

In [2]:
with MPRester() as mpr:  # insert your Materials Project API key here if it's not stored in .pmgrc.yaml
    entries = mpr.get_entries_in_chemsys("Bi-Fe-O-Li-Cl-K")

/Users/mcdermott/opt/miniconda3/envs/rn_dev/lib/python3.9/site-packages/mp_api/client.py:131: builtins.UserWarning: Problem loading MPContribs client: Duplicate operationId: download_entries


Retrieving ThermoDoc documents:   0%|          | 0/714 [00:00<?, ?it/s]

The `GibbsEntrySet` class allows us to automatically convered `ComputedStructureEntry` objects downloaded from the MP database into `GibbsComputedEntry` objects, where DFT-calculated energies have been converted to an AI-estimated equivalent values of the Gibbs free energies of formation, $\Delta G_f$ for all entries at the specified temperature.

For more information, check out the citation in the documentation for `GibbsComputedEntry`.

In [3]:
temp = 900  # units: Kelvin
gibbs_entries = GibbsEntrySet.from_entries(entries, temp)

We can print the entries by calling `.entries` or `.entries_list`:

In [4]:
gibbs_entries.entries_list[0:10]  # first 10 entries

[GibbsComputedEntry | mp-23152 | Bi2 (Bi)
 Gibbs Energy (900 K) = 0.0000,
 GibbsComputedEntry | mp-766035 | Bi12 Cl5 O16 (Bi12Cl5O16)
 Gibbs Energy (900 K) = -36.1620,
 GibbsComputedEntry | mp-32597 | Bi13 O20 (Bi13O20)
 Gibbs Energy (900 K) = -34.7432,
 GibbsComputedEntry | mp-757852 | Bi16 O25 (Bi16O25)
 Gibbs Energy (900 K) = -42.0295,
 GibbsComputedEntry | mp-773884 | Bi38 O60 (Bi19O30)
 Gibbs Energy (900 K) = -100.0888,
 GibbsComputedEntry | mp-560085 | Bi48 Cl20 O62 (Bi24Cl10O31)
 Gibbs Energy (900 K) = -148.3718,
 GibbsComputedEntry | mp-766354 | Bi25 O38 (Bi25O38)
 Gibbs Energy (900 K) = -66.5611,
 GibbsComputedEntry | mp-1206832 | Bi2 Cl2 O1 (Bi2Cl2O)
 Gibbs Energy (900 K) = -0.7448,
 GibbsComputedEntry | mp-37036 | Bi2 O3 (Bi2O3)
 Gibbs Energy (900 K) = -5.0879,
 GibbsComputedEntry | mp-1105780 | Bi8 O12 (Bi2O3)
 Gibbs Energy (900 K) = -20.8975]

The `GibbsEntrySet` class has many helpful functions, such as the following `filter_by_stability()` function, which automatically removes entries which are a specified energy per atom above the convex hull of stability:

In [5]:
filtered_entries = gibbs_entries.filter_by_stability(0.05)

You should now see a much shorter list of entries within the Y-Mn-O system (< 25 meV/atom below the hull)

In [6]:
filtered_entries.entries_list

[GibbsComputedEntry | mp-23152 | Bi2 (Bi)
 Gibbs Energy (900 K) = 0.0000,
 GibbsComputedEntry | mp-766035 | Bi12 Cl5 O16 (Bi12Cl5O16)
 Gibbs Energy (900 K) = -36.1620,
 GibbsComputedEntry | mp-32597 | Bi13 O20 (Bi13O20)
 Gibbs Energy (900 K) = -34.7432,
 GibbsComputedEntry | mp-773884 | Bi38 O60 (Bi19O30)
 Gibbs Energy (900 K) = -100.0888,
 GibbsComputedEntry | mp-560085 | Bi48 Cl20 O62 (Bi24Cl10O31)
 Gibbs Energy (900 K) = -148.3718,
 GibbsComputedEntry | mp-766354 | Bi25 O38 (Bi25O38)
 Gibbs Energy (900 K) = -66.5611,
 GibbsComputedEntry | mp-1190124 | Bi8 O12 (Bi2O3)
 Gibbs Energy (900 K) = -22.0374,
 GibbsComputedEntry | mp-29558 | Bi6 Cl2 O8 (Bi3ClO4)
 Gibbs Energy (900 K) = -18.1487,
 GibbsComputedEntry | mp-651836 | Bi48 Cl24 O60 (Bi4Cl2O5)
 Gibbs Energy (900 K) = -147.5543,
 GibbsComputedEntry | mp-30303 | Bi8 O14 (Bi4O7)
 Gibbs Energy (900 K) = -22.1098,
 GibbsComputedEntry | mp-22908 | Bi4 Cl12 (BiCl3)
 Gibbs Energy (900 K) = -14.0131,
 GibbsComputedEntry | mp-22939 | Bi2 Cl2

## Running enumerators

There are several types of enumerator classes contained within `rxn_network.enumerators`: These are:

1. `BasicEnumerator`: use a combinatorial approach to identify all possible (closed) reactions within a set of entries
2. `BasicOpenEnumerator`: use a combinatorial approach to identify all **open** reactions within a set of entries and a list of specified open entries/elements
3. `MinimizeGibbsEnumerator`: use a thermodynamic approach to identify all reactions within a set of entries that are predicted by minimizing Gibbs free energy between a set of two reacting phases touching at an interface
4. `MinimizeGrandPotentialEnumerator`: use a thermodynamic approach to identify all reactions within a set of entries that are predicted by minimizing the grand potential between a set of two reacting phases touching at an interface with an **open** element at a specified chemical potential

### Basic enumerators
We first create a basic enumerator object by initializing one from the `BasicEnumerator` class:

In [7]:
be = BasicEnumerator()

The `BasicEnumerator` class, as is true for all other enumerator classes, can be provided with several arguments for customizing the enumerator output:

- **precursors**: Optional list of precursor formulas; only reactions which contain at least these phases as reactants will be enumerated.
- **target**: Optional formula of target; only reactions which include formation of this target will be enumerated.
- **calculators**: Optional list of Calculator object names; see calculators module for options (e.g., ["ChempotDistanceCalculator])
- **n**: Maximum reactant/product cardinality; i.e., largest possible number of entries on either side of the reaction. Defaults to 2.
- **exclusive_precursors**: Whether to consider only reactions that have reactants which are a subset of the provided list of precursors. Defaults to True.
- **exclusive_targets**: Whether to consider only reactions that make the provided target directly (i.e. with no byproducts). Defualts to False.
- **remove_unbalanced**: Whether to remove reactions which are unbalanced. Defaults to True.
- **remove_changed**: Whether to remove reactions which can only be balanced by removing a reactant/product or having it change sides. Defaults to True.

Note that the default arguments are good for generating a list of simple (unconstrained) reactions, as we might build for a reaction network. Run the following cell:

In [8]:
all_rxns = be.enumerate(filtered_entries)

Ray is not initialized. Trying with environment variables!
Could not identify existing Ray instance. Creating a new one...


2022-03-30 10:28:34,081	INFO services.py:1374 -- View the Ray dashboard at http://127.0.0.1:8265


[{'NodeID': '99970f334c8bb0c652278c02d430a4ce75921d1ba1b7e4f3cb88bf0d', 'Alive': True, 'NodeManagerAddress': '127.0.0.1', 'NodeManagerHostname': 'MacBook-Pro-3.local', 'NodeManagerPort': 58123, 'ObjectManagerPort': 58122, 'ObjectStoreSocketName': '/tmp/ray/session_2022-03-30_10-28-31_480795_41064/sockets/plasma_store', 'RayletSocketName': '/tmp/ray/session_2022-03-30_10-28-31_480795_41064/sockets/raylet', 'MetricsExportPort': 58417, 'alive': True, 'Resources': {'memory': 10307046606.0, 'CPU': 12.0, 'object_store_memory': 5153523302.0, 'node:127.0.0.1': 1.0}}]


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 55/55 [03:37<00:00,  3.95s/it]


This should complete somewhat quickly (within a few seconds). As a result, a list of ~800 generated reactions will be stored within the `all_rxns` object:

In [9]:
print(len(all_rxns))

46752


In [10]:
pprint(all_rxns[0:10])  # first 10 reactions (may be different on your workstation)

[0.5 K2O + 0.3333 Li5FeO4 -> KLiO + 0.1667 Li4Fe2O5,
 3 FeCl3 + Bi -> BiCl3 + 3 FeCl2,
 Fe2O3 + 0.3333 Li4Bi2O5 -> 0.6667 Fe3O4 + 0.6667 Li2BiO3,
 Bi4Cl2O5 + 0.451 K3BiO3 -> 0.7059 KBi6ClO9 + 0.2157 K3BiCl6,
 BiO2 + 2.333 Li8BiO6 -> 2.667 Li7BiO6 + 0.6667 Bi,
 KFeO2 + Li2BiO3 -> Li2FeO3 + KBiO2,
 Bi13O20 + 1.5 Li8BiO6 -> 2.5 BiO2 + 12 LiBiO2,
 KLi6BiO6 + 0.5 K4Bi2O5 -> 1.5 K2O + Li6Bi2O7,
 LiFeO2 + 0.8 K3FeO3 -> K2LiFeO3 + 0.2 K2Fe4O7,
 Li2O + 0.5 K4Bi2O5 -> K2O + 0.5 Li4Bi2O5]


Looking at the list of reactions above, we see that all reactions are stoichiometrically balanced. If we look at any particular reaction object, we find that the reaction energy and uncertainty can automatically be calculated using the entry energies:

In [11]:
r = all_rxns[0]
print(r)
print(r.energy_per_atom, "±", r.energy_uncertainty_per_atom, "eV/atom")

0.5 K2O + 0.3333 Li5FeO4 -> KLiO + 0.1667 Li4Fe2O5
0.05742336314947239 ± 0.05011876501292629 eV/atom


If we want to generate only reactions which contain specific precursors, we can supply them when we initialize the `BasicEnumerator` object.

In [15]:
be_precursors = BasicEnumerator(precursors=["Y2O3"])
y2o3_rxns_exclusive = be_precursors.enumerate(filtered_entries)

In [16]:
y2o3_rxns_exclusive

[0.6667 Y2O3 -> O2 + 1.333 Y]

Note that by default, this only produces reactions which exclusively have the provided precursor(s). To include reactions that contain this precursor (non-exclusively), set the `exclusive_precursors=False`:

In [17]:
be_precursors = BasicEnumerator(precursors=["Y2O3"], exclusive_precursors=False)
y2o3_rxns = be_precursors.enumerate(filtered_entries)

In [18]:
y2o3_rxns

[Y2O3 + 0.75 Mn3O4 -> 2 YMnO3 + 0.25 Mn,
 1.308 Mn2O3 + 1.026 Y2O3 -> Y2Mn2O7 + 0.05128 YMn12,
 YMn12 + 6 Y2O3 -> 13 Y + 6 Mn2O3,
 Mn + 0.03846 Y2O3 -> 0.07692 YMn12 + 0.03846 Mn2O3,
 2.5 Mn2O3 + 0.1667 Y2O3 -> Mn5O8 + 0.3333 Y,
 Mn5O8 + 0.5 Y2O3 -> YMn2O5 + 1.5 Mn2O3,
 0.01905 Y2O3 + 0.4857 Mn3O4 -> MnO2 + 0.0381 YMn12,
 0.08571 YMn12 + 0.4571 Y2O3 -> Y + 0.3429 Mn3O4,
 MnO + 0.5 Y2O3 -> 0.5 YMn2O5 + 0.5 Y,
 5 Mn + 2.667 Y2O3 -> Mn5O8 + 5.333 Y,
 Mn + 0.6667 Y2O3 -> MnO2 + 1.333 Y,
 MnO + 0.6667 Y2O3 -> YMnO3 + 0.3333 Y,
 Y2O3 + 0.6 Mn3O4 -> 1.8 YMnO3 + 0.2 Y,
 YMn2O5 + 0.375 Y2O3 -> 0.25 Mn + 0.875 Y2Mn2O7,
 MnO + 0.1667 Y2O3 -> 0.3 YMn2O5 + 0.03333 YMn12,
 1.5 Mn2O3 + 0.5 Y2O3 -> YMn2O5 + MnO,
 0.5 Mn2O3 + 0.1667 Y2O3 -> MnO2 + 0.3333 Y,
 1.889 MnO + 0.03704 Y2O3 -> MnO2 + 0.07407 YMn12,
 0.6154 Mn2O3 + 0.05128 Y2O3 -> O2 + 0.1026 YMn12,
 Mn5O8 + 2 Y2O3 -> Mn + 2 Y2Mn2O7,
 MnO2 + 0.25 Y2O3 -> 0.125 O2 + 0.5 YMn2O5,
 7.667 MnO + 0.1111 Y2O3 -> Mn5O8 + 0.2222 YMn12,
 Mn5O8 + 2.069 Y2O

This same approach can be used for the target phase(s) as well.

With `exclusive_targets=False`:

In [41]:
be_target = BasicEnumerator(targets=["YMnO3"])
ymno3_rxns = be_target.enumerate(filtered_entries)

189.5
Parallelizing enumeration for 8 chemical systems


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:18<00:00,  2.34s/it]


In [42]:
ymno3_rxns

[Li5FeO4 + 0.25 FeBi25O39 -> 1.25 FeBiO3 + 5 LiBiO2,
 FeCl3 + 0.25 K3BiO3 -> 0.25 FeBiO3 + 0.75 KFeCl4,
 0.04 FeBi25O39 + 0.6 K17Fe5O16 -> FeBiO3 + 2.04 K5FeO4,
 2.333 K5FeO4 + 0.5 Fe4Bi2O9 -> FeBiO3 + 0.8333 K14Fe4O13,
 0.1667 K3BiCl6 + 0.1667 K3FeO3 -> KCl + 0.1667 FeBiO3,
 FeCl3 + 3 BiClO -> 2 BiCl3 + FeBiO3,
 Bi3ClO4 + 2 Fe4Bi2O9 -> FeClO + 7 FeBiO3,
 2 Li2FeO2 + LiBiO3 -> FeBiO3 + Li5FeO4,
 0.5 Bi2O3 + K5FeO4 -> FeBiO3 + 2.5 K2O,
 0.04 FeBi25O39 + 0.288 K2Fe4O7 -> FeBiO3 + 0.192 K3FeO3,
 Bi2O3 + Fe2O3 -> 2 FeBiO3,
 Bi13O20 + 5.5 Fe4Bi2O9 -> 0.5 Bi4O7 + 22 FeBiO3,
 Li2FeO3 + 1.5 LiBiO2 -> FeBiO3 + 0.5 Li7BiO6,
 K2O + Fe4Bi2O9 -> 2 KFeO2 + 2 FeBiO3,
 FeO + 0.5455 Bi4O7 -> FeBiO3 + 0.09091 Bi13O20,
 0.5 K4Bi2O5 + 0.2 K17Fe5O16 -> FeBiO3 + 2.7 K2O,
 0.75 Fe4Bi2O9 + 0.25 K4Bi2O5 -> KFeO2 + 2 FeBiO3,
 Bi19O30 + 6.5 Fe4Bi2O9 -> 1.5 Bi4O7 + 26 FeBiO3,
 Bi2O3 + 0.1818 K14Fe4O13 -> 0.7273 FeBiO3 + 0.6364 K4Bi2O5,
 2 BiO2 + K3FeO3 -> FeBiO3 + K3BiO4,
 Bi4Cl2O5 + 0.08333 FeBi25O39 -> 0.08333 

With `exclusive_targets=True`:

In [21]:
be_target = BasicEnumerator(targets=["YMnO3"], exclusive_targets=True)
ymno3_rxns = be_target.enumerate(filtered_entries)

In [22]:
ymno3_rxns

[Mn2O3 + Y2O3 -> 2 YMnO3]

And finally with multiple targets (e.g., YMnO3 and O2):

In [23]:
be_targets = BasicEnumerator(targets=["YMnO3", "O2"], exclusive_targets=True)
ymno3_rxns_o2 = be_targets.enumerate(filtered_entries)

In [24]:
ymno3_rxns_o2

[0.5 Y2Mn2O7 -> YMnO3 + 0.25 O2,
 Mn2O3 + Y2O3 -> 2 YMnO3,
 MnO2 + 0.5 Y2O3 -> YMnO3 + 0.25 O2,
 YMn2O5 + 0.5 Y2O3 -> 2 YMnO3 + 0.25 O2,
 Mn5O8 + 2.5 Y2O3 -> 5 YMnO3 + 0.25 O2]

#### Open entries

In the previous cell, we showed that it was possible to specify YMnO3 as a target, along with O2. However, because O2 is a gas, it is often desirable to include it as an **open entry** in addition to the 1-2 possible precursors/targets. For example, we may want to specify a reaction that follows:

$$ A + B ~ (+~O_2) \rightarrow C + D ~ (+~O_2) $$

To do this, we use the `BasicOpenEnumerator` class, which is an extension to the previous basic enumerator. All of the lessons learned above also apply to this class, although now a list of open entry formulas must be specified.

In [53]:
be_target_open = BasicOpenEnumerator(["O2"],targets=["YMnO3"])
ymno3_rxns_open = be_target_open.enumerate(filtered_entries)

Mn-O-Y: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.41s/it]


In [54]:
ymno3_rxns_open

[O2 + Y2O3 + 1.5 Mn3O4 -> 2 YMnO3 + 2.5 MnO2,
 YMn2O5 + 0.8571 O2 + Y -> 0.5714 YMnO3 + 0.7143 Y2Mn2O7,
 5 MnO + YMn12 -> YMnO3 + 16 Mn + O2,
 2 Y2Mn2O7 + 3 MnO2 -> YMnO3 + O2 + 3 YMn2O5,
 O2 + MnO + 0.07692 YMn12 -> 0.07692 YMnO3 + 0.9231 Mn2O3,
 7.5 O2 + YMn12 + Y2O3 -> 3 YMnO3 + 9 MnO,
 0.5758 YMn2O5 + 0.1818 O2 + 0.4242 Y -> YMnO3 + 0.0303 Mn5O8,
 1.143 O2 + MnO + Y -> 0.4286 YMnO3 + 0.2857 Y2Mn2O7,
 O2 + 0.2857 Mn5O8 + 1.429 Y -> 1.429 YMnO3,
 1.1 YMn2O5 -> YMnO3 + 1.25 O2 + 0.1 YMn12,
 O2 + Mn5O8 + 0.1282 YMn12 -> 0.1282 YMnO3 + 3.205 Mn2O3,
 O2 + MnO2 + 0.1509 YMn12 -> 0.1509 YMnO3 + 0.8868 Mn3O4,
 0.5 Y2Mn2O7 + 0.2 Mn5O8 -> YMnO3 + Mn + 1.05 O2,
 YMn2O5 -> YMnO3 + 0.25 O2 + 0.5 Mn2O3,
 Mn + 1.667 Y2O3 -> YMnO3 + O2 + 2.333 Y,
 0.6667 O2 + MnO2 + Y -> 0.3333 YMnO3 + 0.3333 Y2Mn2O7,
 0.5 Y2Mn2O7 + 0.5 Mn2O3 -> YMnO3 + Mn + O2,
 0.5 Y2Mn2O7 + 3 Mn5O8 -> YMnO3 + O2 + 7.5 Mn2O3,
 O2 + Y + 0.5 Mn3O4 -> YMnO3 + 0.5 MnO2,
 0.5 Y2Mn2O7 + 0.6667 Mn3O4 -> YMnO3 + 0.08333 O2 + Mn2O3,
 YMn2

You will now see that the reactions from before are included in this new list, along with many other open-O2 reactions -- some which even have 3 reactants or 3 products. In other words, the open entry/entries do not count towards the specified $n$ of the reactions.

It is even possible to specify multiple open entries, allowing for even more complex reactions (such as ones with 4+ reactants or products). For example, specifiyng Y2O3 and O2 as both being open:

In [55]:
be_target_open2 = BasicOpenEnumerator(["Y2O3", "O2"],targets=["YMnO3"])
ymno3_rxns_open2 = be_target_open2.enumerate(filtered_entries)

Mn-O-Y: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.02s/it]


In [57]:
ymno3_rxns_open2

[0.2 YMn2O5 + 1.1 Y2Mn2O7 -> YMnO3 + O2 + 1.6 MnO + 0.7 Y2O3,
 O2 + 13 Y2Mn2O7 + YMn12 + Y2O3 -> 29 YMnO3 + 9 MnO,
 YMn2O5 + 0.8571 O2 + Y -> 0.5714 YMnO3 + 0.7143 Y2Mn2O7,
 5 MnO + YMn12 -> YMnO3 + 16 Mn + O2,
 0.4544 YMn2O5 + 0.452 Y2Mn2O7 -> YMnO3 + O2 + 0.06773 YMn12 + 0.1453 Y2O3,
 3.5 YMn2O5 + Mn + Y2O3 -> 5.5 YMnO3 + 0.5 Mn5O8,
 2 Y2Mn2O7 + 3 MnO2 -> YMnO3 + O2 + 3 YMn2O5,
 O2 + MnO + 0.07692 YMn12 -> 0.07692 YMnO3 + 0.9231 Mn2O3,
 O2 + YMn12 + 13 Y2O3 + Mn3O4 -> 15 YMnO3 + 12 Y,
 0.5758 YMn2O5 + 0.1818 O2 + 0.4242 Y -> YMnO3 + 0.0303 Mn5O8,
 1.143 O2 + MnO + Y -> 0.4286 YMnO3 + 0.2857 Y2Mn2O7,
 O2 + 0.09709 YMn12 + Mn2O3 + Y2O3 -> 2.097 YMnO3 + 0.2136 Mn5O8,
 O2 + 0.2857 Mn5O8 + 1.429 Y -> 1.429 YMnO3,
 1.1 YMn2O5 -> YMnO3 + 1.25 O2 + 0.1 YMn12,
 O2 + Mn5O8 + 0.1282 YMn12 -> 0.1282 YMnO3 + 3.205 Mn2O3,
 27 MnO2 + YMn12 + Y2O3 -> 3 YMnO3 + 12 Mn3O4,
 3 YMn2O5 + 5 Mn2O3 -> YMnO3 + 3 Mn5O8 + Y2O3,
 O2 + MnO2 + 0.1509 YMn12 -> 0.1509 YMnO3 + 0.8868 Mn3O4,
 0.5 Y2Mn2O7 + 0.2 Mn5O8 -

This may be useful for systems where more than 1 gaseous, liquid, or molten phases are available to the reacting system, such as $O_2$ and a molten salt (e.g. $LiCl$)

### Minimize Enumerators

The "minimize" enumerators produce reactions via a thermodynamic approach, rather than a purely combinatorial one. This means that reactions are produced from a phase diagram approach, where a new convex hull is drawn connecting two compositions within a closed (Gibbs) or open (Grand Potential) system. See the `InterfacialReactivity` class within the _pymatgen_ package for more information.

It is **important to note** that reactions produced with the minimize enumerators may overlap some with the basic enumerators, but the minimize enumerators have the restriction that all original reactions produced must have a negative reaction energy and result in a set of product phases which are stable with respect to each other (i.e. they share a facet of the phase diagram).

Even though these enumerators also yield the reverse reactions to those described above, it is impossible for them to identify a reaction where both sides of the reaction contain phases that are unstable with respect to each other. In other words, every reaction proudced has either reactants that share a phase diagram facet or products that share a phase diagram facet).

The `MinimizeGibbsEnumerator` contains similar arguments as the basic enumerators. See the docstrings for updated information:
- **precursors**: Optional formulas of precursors.
- **targets**: Optional formulas of targets; only reactions which make these targets
    will be enumerated.
- **calculators**: Optional list of Calculator object names; see calculators
    module for options (e.g., ["ChempotDistanceCalculator"])
- **exclusive_precursors**: Whether to consider only reactions that have
    reactants which are a subset of the provided list of precursors.
    Defaults to True.
- **exclusive_targets**: Whether to consider only reactions that make the
    provided target directly (i.e. with no byproducts). Defualts to False.

In [58]:
mge = MinimizeGibbsEnumerator()

The default arguments, as before, help produce all reactions in a set of entries, given minimal constraints.

In [59]:
rxns = mge.enumerate(filtered_entries)

Mn-Y: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00,  6.68it/s]


In [60]:
pprint(rxns[0:10])

[MnO2 + 0.5 Y2Mn2O7 -> YMn2O5 + 0.25 O2,
 0.5 Y + 0.5 YMn2O5 -> MnO + 0.5 Y2O3,
 0.5 Mn2O3 + 0.3333 Y -> MnO + 0.1667 Y2O3,
 0.3333 Mn3O4 + 0.8889 Y -> Mn + 0.4444 Y2O3,
 0.1304 Mn5O8 + 0.02899 YMn12 -> MnO + 0.01449 Y2O3,
 0.3333 Y + YMnO3 -> MnO + 0.6667 Y2O3,
 2 MnO2 + 0.5 Y2O3 -> YMn2O5 + 0.25 O2,
 0.02564 YMn12 + 0.3462 Mn2O3 -> MnO + 0.01282 Y2O3,
 0.2857 Y + 0.7143 YMn2O5 -> YMnO3 + 0.1429 Mn3O4,
 Mn5O8 + 2 Y2Mn2O7 -> MnO2 + 4 YMn2O5]


And as before, we can specify various combinations of precursors and targets, as well as whether or not they should be "exclusive".

In [61]:
mge_precursors = MinimizeGibbsEnumerator(precursors=["Y2O3"], exclusive_precursors=False)
rxns = mge_precursors.enumerate(filtered_entries)
rxns

O-Y: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 21.50it/s]


[2.5 Mn2O3 + 0.5 Y2O3 -> YMn2O5 + Mn3O4,
 0.5 Mn2O3 + 0.5 Y2O3 -> YMnO3,
 0.625 Mn5O8 + 0.5 Y2O3 -> YMn2O5 + 0.375 Mn3O4,
 0.5 Mn3O4 + 0.5 Y2O3 -> YMnO3 + 0.5 MnO,
 0.6667 Mn5O8 + 0.3333 Y2O3 -> Mn2O3 + 0.6667 YMn2O5,
 2 MnO2 + Y2O3 -> Y2Mn2O7,
 2 MnO2 + 0.5 Y2O3 -> YMn2O5 + 0.25 O2,
 0.3333 Mn5O8 + 0.6667 Y2O3 -> YMnO3 + 0.3333 YMn2O5]

In [62]:
mge_precursors = MinimizeGibbsEnumerator(precursors=["Y2O3", "Mn2O3"])
rxns = mge_precursors.enumerate(filtered_entries)
rxns

Mn-O-Y: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 43.90it/s]


[0.5 Mn2O3 + 0.5 Y2O3 -> YMnO3, 2.5 Mn2O3 + 0.5 Y2O3 -> YMn2O5 + Mn3O4]

In [63]:
mge_targets = MinimizeGibbsEnumerator(targets=["YMnO3"], exclusive_targets=True)
rxns = mge_targets.enumerate(filtered_entries)
rxns

Mn-O-Y: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.37it/s]


[0.5 Mn2O3 + 0.5 Y2O3 -> YMnO3]

In [64]:
mge_targets = MinimizeGibbsEnumerator(targets=["YMnO3"], exclusive_targets=False)
rxns = mge_targets.enumerate(filtered_entries)
rxns

Mn-O-Y: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.53it/s]


[0.5 Mn + 0.5 Y2Mn2O7 -> YMnO3 + 0.5 MnO,
 MnO + Y2Mn2O7 -> YMnO3 + YMn2O5,
 YMn12 + 7 O2 -> YMnO3 + 11 MnO,
 Mn + YMn2O5 -> YMnO3 + 2 MnO,
 5 Mn2O3 + Y -> YMnO3 + 3 Mn3O4,
 14 MnO2 + YMn12 -> YMnO3 + 25 MnO,
 0.3333 Mn + 0.6667 Y2Mn2O7 -> YMnO3 + 0.3333 YMn2O5,
 0.2857 Y + 0.7143 YMn2O5 -> YMnO3 + 0.1429 Mn3O4,
 0.375 Mn + 0.5 Y2Mn2O7 -> YMnO3 + 0.125 Mn3O4,
 YMn12 + 14 Mn2O3 -> YMnO3 + 39 MnO,
 1.25 Mn5O8 + Y -> YMnO3 + 1.75 Mn3O4,
 YMn12 + 14 Mn3O4 -> YMnO3 + 53 MnO,
 1.5 MnO + 0.5 Y2Mn2O7 -> YMnO3 + 0.5 Mn3O4,
 0.06667 YMn12 + 0.9333 YMn2O5 -> YMnO3 + 1.667 MnO,
 2 MnO2 + Y -> YMnO3 + MnO,
 0.3333 Mn5O8 + 0.6667 Y2O3 -> YMnO3 + 0.3333 YMn2O5,
 0.5 Mn3O4 + 0.5 Y2O3 -> YMnO3 + 0.5 MnO,
 0.3333 Y + 0.6667 YMn2O5 -> YMnO3 + 0.3333 MnO,
 0.03448 YMn12 + 0.4828 Y2Mn2O7 -> YMnO3 + 0.3793 MnO,
 2 MnO + YMn2O5 -> YMnO3 + Mn3O4,
 2 Mn2O3 + Y -> YMnO3 + 3 MnO,
 2 Mn3O4 + Y -> YMnO3 + 5 MnO,
 26.5 MnO2 + YMn12 -> YMnO3 + 12.5 Mn3O4,
 0.5 Mn + YMn2O5 -> YMnO3 + 0.5 Mn3O4,
 0.025 YMn12 + 0.625 Y

#### Open entries

And once again, as before, we can do all the same analysis with open entries. This time, the grand potential is used as the thermodynamic free energy which is minimized:

$$ \Phi = G - \mu_iN_i $$

Where $i$ is the open species with chemical potential $\mu_i$ with a molar amount $N_i$.

In [65]:
mgpe = MinimizeGrandPotentialEnumerator(open_elem=Element("O"), mu=0)
open_rxns = mgpe.enumerate(filtered_entries)
open_rxns

O-Y: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  7.45it/s]


[4 MnO2 + 2 Y2Mn2O7 -> O2 + 4 YMn2O5 (mu_O=0),
 2 MnO + Y2O3 + O2 -> Y2Mn2O7 (mu_O=0),
 0.5 Mn2O3 + 0.25 O2 -> MnO2 (mu_O=0),
 2 Mn + Y2O3 + 2 O2 -> Y2Mn2O7 (mu_O=0),
 2 MnO2 + 2 Y + 1.5 O2 -> Y2Mn2O7 (mu_O=0),
 Mn + 0.5 Y2Mn2O7 + 0.75 O2 -> YMn2O5 (mu_O=0),
 2 MnO + 2 Y + 2.5 O2 -> Y2Mn2O7 (mu_O=0),
 0.6667 Mn3O4 + Y2O3 + 0.6667 O2 -> Y2Mn2O7 (mu_O=0),
 0.5 Y2O3 + YMn2O5 + 0.25 O2 -> Y2Mn2O7 (mu_O=0),
 0.6667 Mn3O4 + 2 Y + 2.167 O2 -> Y2Mn2O7 (mu_O=0),
 0.3333 Mn3O4 + 0.5 Y2Mn2O7 + 0.08333 O2 -> YMn2O5 (mu_O=0),
 0.3333 Mn3O4 + YMnO3 + 0.3333 O2 -> YMn2O5 (mu_O=0),
 0.4 Mn5O8 + 2 Y + 1.9 O2 -> Y2Mn2O7 (mu_O=0),
 8 MnO2 + 2 Y2O3 -> O2 + 4 YMn2O5 (mu_O=0),
 0.09091 YMn12 + 0.9091 YMnO3 + 1.136 O2 -> YMn2O5 (mu_O=0),
 0.1667 YMn12 + 0.9167 Y2O3 + 2.125 O2 -> Y2Mn2O7 (mu_O=0),
 0.4 Mn5O8 + Y + 0.9 O2 -> YMn2O5 (mu_O=0),
 MnO + 0.5 O2 -> MnO2 (mu_O=0),
 0.4 Mn5O8 + 0.5 Y2O3 + 0.15 O2 -> YMn2O5 (mu_O=0),
 0.2 Mn5O8 + YMnO3 + 0.2 O2 -> YMn2O5 (mu_O=0),
 2 MnO2 + Y + 0.5 O2 -> YMn2O5 (mu_O=0)

Note that the reaction objects returned are now of a different type: `OpenComputedReaction`. This class allows for easy specification of reactions where one of the elements is assigned a chemical potential.

In [66]:
r = open_rxns[0]
print(r.__class__.__name__)

OpenComputedReaction


We can also, as before, customize which precursors and targets are specified:

In [67]:
mgpe_precursors = MinimizeGrandPotentialEnumerator(open_elem=Element("O"), mu=0,
                                                   precursors=["Y2O3"], exclusive_precursors=False)
open_rxns_precursors = mgpe_precursors.enumerate(filtered_entries)
open_rxns_precursors

O-Y: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 16.24it/s]


[0.4 Mn5O8 + Y2O3 + 0.4 O2 -> Y2Mn2O7 (mu_O=0),
 0.4 Mn5O8 + 0.5 Y2O3 + 0.15 O2 -> YMn2O5 (mu_O=0),
 2 MnO + 0.5 Y2O3 + 0.75 O2 -> YMn2O5 (mu_O=0),
 2 MnO + Y2O3 + O2 -> Y2Mn2O7 (mu_O=0),
 2 Mn + Y2O3 + 2 O2 -> Y2Mn2O7 (mu_O=0),
 0.6667 Mn3O4 + 0.5 Y2O3 + 0.4167 O2 -> YMn2O5 (mu_O=0),
 0.6667 Mn3O4 + Y2O3 + 0.6667 O2 -> Y2Mn2O7 (mu_O=0),
 Mn2O3 + Y2O3 + 0.5 O2 -> Y2Mn2O7 (mu_O=0),
 0.5 Y2O3 + YMn2O5 + 0.25 O2 -> Y2Mn2O7 (mu_O=0),
 0.1667 YMn12 + 0.4167 Y2O3 + 1.875 O2 -> YMn2O5 (mu_O=0),
 2 Mn + 0.5 Y2O3 + 1.75 O2 -> YMn2O5 (mu_O=0),
 Mn2O3 + 0.5 Y2O3 + 0.25 O2 -> YMn2O5 (mu_O=0),
 8 MnO2 + 2 Y2O3 -> O2 + 4 YMn2O5 (mu_O=0),
 0.1667 YMn12 + 0.9167 Y2O3 + 2.125 O2 -> Y2Mn2O7 (mu_O=0)]

In [68]:
mgpe_precursors = MinimizeGrandPotentialEnumerator(open_elem=Element("O"), mu=0,
                                                   targets=["Y2Mn2O7"])
open_rxns_targets = mgpe_precursors.enumerate(filtered_entries)
open_rxns_targets

Mn-O-Y: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.07it/s]


[0.4 Mn5O8 + Y2O3 + 0.4 O2 -> Y2Mn2O7 (mu_O=0),
 2 MnO + Y2O3 + O2 -> Y2Mn2O7 (mu_O=0),
 2 Mn + Y2O3 + 2 O2 -> Y2Mn2O7 (mu_O=0),
 2 MnO2 + 2 Y + 1.5 O2 -> Y2Mn2O7 (mu_O=0),
 0.6667 Mn3O4 + 2 Y + 2.167 O2 -> Y2Mn2O7 (mu_O=0),
 2 MnO + 2 Y + 2.5 O2 -> Y2Mn2O7 (mu_O=0),
 0.6667 Mn3O4 + Y2O3 + 0.6667 O2 -> Y2Mn2O7 (mu_O=0),
 Mn2O3 + Y2O3 + 0.5 O2 -> Y2Mn2O7 (mu_O=0),
 0.5 Y2O3 + YMn2O5 + 0.25 O2 -> Y2Mn2O7 (mu_O=0),
 Y + YMn2O5 + O2 -> Y2Mn2O7 (mu_O=0),
 0.4 Mn5O8 + 2 Y + 1.9 O2 -> Y2Mn2O7 (mu_O=0),
 Mn2O3 + 2 Y + 2 O2 -> Y2Mn2O7 (mu_O=0),
 2 YMnO3 + 0.5 O2 -> Y2Mn2O7 (mu_O=0),
 0.1667 YMn12 + 0.9167 Y2O3 + 2.125 O2 -> Y2Mn2O7 (mu_O=0)]

Note that setting the chemical potential to a value outside of the range of stability of the target causes the enumerator to yield no reactions:

In [69]:
mgpe_precursors = MinimizeGrandPotentialEnumerator(open_elem=Element("O"), mu=-3,
                                                   targets=["Y2Mn2O7"])
open_rxns_targets = mgpe_precursors.enumerate(filtered_entries)
open_rxns_targets

Mn-O-Y: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  4.03it/s]


[]

## Running enumerators with the Fireworks package

A convenient way to run one or more enumerators is to use the `EnumeratorFW`, a wrapper for running enumerators that is written as a `Firework` within the _fireworks_ package. This allows for easy workflow management, such as queue submissions, I/O serialization, tracking of calculations, etc.

The `EnumeratorFW` has support for acquiring entries either from a custom entry database (MongoDB), or from the Materials Project using `MPRester` API. To set up the firework, we simply provide a list of enumerators, and any of the optional parameters. If a chemical system is provided, entires for that system will be acquired before running the enumerator(s).

In [50]:
from rxn_network.fireworks import EnumeratorFW
from fireworks import Workflow, LaunchPad

2022-03-07 15:48:43,241	INFO worker.py:862 -- Calling ray.init() again after it has already been called.


In [51]:
be_target = BasicEnumerator(targets=["YMnO3"])
fw = EnumeratorFW([be_target], chemsys="Y-Mn-O-Li-Na-Cl-F", entry_set_params={"e_above_hull":0})

The firework is simply a dictionary with a list of FireTasks where 1) entries are acquired, 2) enumerators are run, and 3) reactions are stored in a specified database

We then place the firework by itself into a `Workflow` so that it can be added to our fireworks `LaunchPad` (see the fireworks documentation for more info on setting this up):

In [52]:
wf = Workflow([fw], name="Y-Mn-O Enumerator")

In [53]:
lpad = LaunchPad.auto_load()
lpad.add_wf(wf)

2022-03-07 15:48:54,840 INFO Added a workflow. id_map: {-3: 5415}


{-3: 5415}

The workflow can now either be launched through a queue submission (i.e. for supercomputer use) or simply on your local machine, by running `rlaunch singleshot` in your terminal as below. **Note**: this will launch in whatever folder this notebook is in!

In [79]:
!rlaunch singleshot

2022-02-16 13:08:22,493 INFO Hostname/IP lookup (this will take a few seconds)
2022-02-16 13:08:22,496 INFO Launching Rocket
2022-02-16 13:08:26,918 INFO RUNNING fw_id: 4618 in directory: /Users/mcdermott/PycharmProjects/reaction-network/notebooks
2022-02-16 13:08:27,164 INFO Task started: {{rxn_network.firetasks.build_inputs.EntriesFromDb}}.
2022-02-16 13:08:28,412 INFO Task completed: {{rxn_network.firetasks.build_inputs.EntriesFromDb}} 
2022-02-16 13:08:28,474 INFO Task started: {{rxn_network.firetasks.run_calc.RunEnumerators}}.
O-Y: 100%|████████████████████████████████████████| 4/4 [00:01<00:00,  3.56it/s]
2022-02-16 13:08:29,734 INFO Task completed: {{rxn_network.firetasks.run_calc.RunEnumerators}} 
2022-02-16 13:08:29,830 INFO Task started: {{rxn_network.firetasks.parse_outputs.ReactionsToDb}}.
2022-02-16 13:08:30,169 INFO rxn_network.utils.database Updating /Users/mcdermott/PycharmProjects/reaction-network/notebooks with taskid = 82
2022-02-16 13:08:30,291 INFO Task completed: 

If any errors are encountered, please double-check that your Fireworks settings are configured properly. If the error persists, please raise an Issue here: https://github.com/GENESIS-EFRC/reaction-network/issues